In [ ]:
import requests
import time
import pandas as pd
from datetime import datetime, timedelta
import os
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
BASE_URL = "https://ows.goszakup.gov.kz"
API_ENDPOINT = "/v3/plans/deleted"
FINAL_FILE = "goszakup_deleted_data.parquet"
TEMP_DIR = "temp_parquet_files"

# Создаём директорию для временных файлов
if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}
next_page = f"{API_ENDPOINT}?limit=500"
seen_ids = set()

# Диапазон дат
START_DATE = datetime.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
STOP_DATE = datetime.now()

# Параметры для остановки
MAX_EMPTY_PAGES = 10
empty_page_count = 0
temp_file_count = 0
BATCH_SIZE = 10000

def parse_date(date_str):
    """Парсит дату, поддерживая оба формата."""
    try:
        return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f")
    except ValueError:
        try:
            return datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        except ValueError as e:
            raise ValueError(f"Неподдерживаемый формат даты: {date_str}") from e

def save_batch(filtered_items, temp_file_count):
    """Сохраняет батч данных во временный файл."""
    temp_file = os.path.join(TEMP_DIR, f"temp_batch_{temp_file_count}.parquet")
    df = pd.DataFrame(filtered_items)
    df.to_parquet(temp_file, index=False, engine="pyarrow")
    logging.info(f"Сохранён временный файл: {temp_file} с {len(filtered_items)} записями")
    return temp_file_count + 1

def merge_temp_files(final_file):
    """Объединяет временные файлы в итоговый."""
    temp_files = [os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR) if f.endswith(".parquet")]
    if not temp_files:
        logging.info("Нет данных для объединения.")
        return
    
    combined_df = pd.concat([pd.read_parquet(f) for f in temp_files], ignore_index=True)
    combined_df.to_parquet(final_file, index=False, engine="pyarrow")
    logging.info(f"Все данные объединены в {final_file}, всего записей: {len(combined_df)}")
    
    for f in temp_files:
        os.remove(f)
    os.rmdir(TEMP_DIR)

filtered_items = []
total_records = 0

while next_page:
    try:
        response = requests.get(f"{BASE_URL}{next_page}", headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            logging.info("Загрузка завершена, новых данных нет.")
            break

        logging.info(f"Получено {len(items)} записей на странице.")
        page_filtered_items = []  # Отдельный список для записей текущей страницы
        page_has_relevant_data = False

        for item in items:
            try:
                item_date = parse_date(item["index_date"])
                if START_DATE <= item_date <= STOP_DATE:
                    if item["id"] not in seen_ids:
                        page_filtered_items.append(item)
                        filtered_items.append(item)
                        seen_ids.add(item["id"])
                        page_has_relevant_data = True
                else:
                    logging.debug(f"Пропущена запись ID {item['id']} с датой {item_date} (вне диапазона)")
            except ValueError as e:
                logging.warning(f"Ошибка парсинга даты для записи {item['id']}: {e}")
                continue

        if page_has_relevant_data:
            empty_page_count = 0
            total_records += len(page_filtered_items)
            logging.info(f"Добавлено {len(page_filtered_items)} записей с этой страницы. Всего: {total_records}")
        else:
            empty_page_count += 1
            logging.info(f"На этой странице нет записей в диапазоне дат. Пустых страниц подряд: {empty_page_count}")

        if len(filtered_items) >= BATCH_SIZE:
            temp_file_count = save_batch(filtered_items, temp_file_count)
            filtered_items = []

        if empty_page_count >= MAX_EMPTY_PAGES:
            logging.info(f"Достигнут лимит пустых страниц ({MAX_EMPTY_PAGES}), завершаем загрузку.")
            break

        next_page = data.get("next_page")
        if not next_page:
            logging.info("Достигнута последняя страница, завершаем загрузку.")
            break

        time.sleep(5)

    except requests.exceptions.Timeout:
        logging.error("Таймаут запроса, ждём 30 секунд и пробуем снова...")
        time.sleep(30)
        continue
    except requests.exceptions.ConnectionError as e:
        logging.error(f"Ошибка соединения: {e}, ждём 30 секунд и пробуем снова...")
        time.sleep(30)
        continue
    except requests.exceptions.RequestException as e:
        logging.error(f"Неизвестная ошибка запроса: {e}, ждём 30 секунд и пробуем снова...")
        time.sleep(30)
        continue

# Сохраняем оставшиеся данные
if filtered_items:
    temp_file_count = save_batch(filtered_items, temp_file_count)

# Объединяем временные файлы
merge_temp_files(FINAL_FILE)

if total_records == 0:
    logging.info("Нет данных для сохранения в указанном диапазоне дат.")

2025-03-28 14:11:15,030 - ERROR - Ошибка соединения: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out., ждём 30 секунд и пробуем снова...
2025-03-28 14:11:45,623 - INFO - Получено 500 записей на странице.
2025-03-28 14:11:45,647 - INFO - Добавлено 500 записей с этой страницы. Всего: 500
2025-03-28 14:11:51,109 - INFO - Получено 500 записей на странице.
2025-03-28 14:11:51,121 - INFO - Добавлено 500 записей с этой страницы. Всего: 1000
2025-03-28 14:11:56,531 - INFO - Получено 500 записей на странице.
2025-03-28 14:11:56,556 - INFO - Добавлено 500 записей с этой страницы. Всего: 1500
2025-03-28 14:12:01,960 - INFO - Получено 500 записей на странице.
2025-03-28 14:12:01,973 - INFO - Добавлено 500 записей с этой страницы. Всего: 2000
2025-03-28 14:12:07,596 - INFO - Получено 500 записей на странице.
2025-03-28 14:12:07,609 - INFO - Добавлено 500 записей с этой страницы. Всего: 2500
2025-03-28 14:12:13,475 - INFO - Получено 500 записей на странице.
2025-03-28 14:12

2025-03-28 14:18:23,730 - INFO - Добавлено 500 записей с этой страницы. Всего: 24987
2025-03-28 14:18:29,140 - INFO - Получено 500 записей на странице.
2025-03-28 14:18:29,154 - INFO - Добавлено 500 записей с этой страницы. Всего: 25487
2025-03-28 14:18:34,671 - INFO - Получено 500 записей на странице.
2025-03-28 14:18:34,684 - INFO - Добавлено 500 записей с этой страницы. Всего: 25987
2025-03-28 14:18:40,144 - INFO - Получено 500 записей на странице.
2025-03-28 14:18:40,158 - INFO - Добавлено 500 записей с этой страницы. Всего: 26487
2025-03-28 14:18:45,656 - INFO - Получено 500 записей на странице.
2025-03-28 14:18:45,680 - INFO - Добавлено 499 записей с этой страницы. Всего: 26986
2025-03-28 14:18:51,155 - INFO - Получено 500 записей на странице.
2025-03-28 14:18:51,172 - INFO - Добавлено 500 записей с этой страницы. Всего: 27486
2025-03-28 14:19:26,278 - ERROR - Таймаут запроса, ждём 30 секунд и пробуем снова...
2025-03-28 14:19:56,944 - INFO - Получено 500 записей на странице.
202

2025-03-28 14:26:11,993 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:12,006 - INFO - Добавлено 484 записей с этой страницы. Всего: 50438
2025-03-28 14:26:17,611 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:17,624 - INFO - Добавлено 499 записей с этой страницы. Всего: 50937
2025-03-28 14:26:23,155 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:23,176 - INFO - Добавлено 500 записей с этой страницы. Всего: 51437
2025-03-28 14:26:28,584 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:28,600 - INFO - Добавлено 499 записей с этой страницы. Всего: 51936
2025-03-28 14:26:34,021 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:34,034 - INFO - Добавлено 489 записей с этой страницы. Всего: 52425
2025-03-28 14:26:34,080 - INFO - Сохранён временный файл: temp_parquet_files\temp_batch_4.parquet с 10453 записями
2025-03-28 14:26:39,543 - INFO - Получено 500 записей на странице.
2025-03-28 14:26:39,556 - INFO - Добавлено 499 записей с э

2025-03-28 14:34:39,428 - INFO - Добавлено 500 записей с этой страницы. Всего: 74365
2025-03-28 14:34:44,930 - INFO - Получено 500 записей на странице.
2025-03-28 14:34:44,942 - INFO - Добавлено 500 записей с этой страницы. Всего: 74865
2025-03-28 14:34:50,517 - INFO - Получено 500 записей на странице.
2025-03-28 14:34:50,538 - INFO - Добавлено 496 записей с этой страницы. Всего: 75361
2025-03-28 14:34:55,938 - INFO - Получено 500 записей на странице.
2025-03-28 14:34:55,952 - INFO - Добавлено 500 записей с этой страницы. Всего: 75861
2025-03-28 14:35:01,491 - INFO - Получено 500 записей на странице.
2025-03-28 14:35:01,506 - INFO - Добавлено 500 записей с этой страницы. Всего: 76361
2025-03-28 14:35:06,904 - INFO - Получено 500 записей на странице.
2025-03-28 14:35:06,916 - INFO - Добавлено 500 записей с этой страницы. Всего: 76861
2025-03-28 14:35:12,489 - INFO - Получено 500 записей на странице.
2025-03-28 14:35:12,508 - INFO - Добавлено 500 записей с этой страницы. Всего: 77361
202

2025-03-28 14:41:25,553 - INFO - Добавлено 498 записей с этой страницы. Всего: 99681
2025-03-28 14:41:31,325 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:31,335 - INFO - Добавлено 500 записей с этой страницы. Всего: 100181
2025-03-28 14:41:36,876 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:36,880 - INFO - Добавлено 500 записей с этой страницы. Всего: 100681
2025-03-28 14:41:42,378 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:42,382 - INFO - Добавлено 500 записей с этой страницы. Всего: 101181
2025-03-28 14:41:48,000 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:48,006 - INFO - Добавлено 500 записей с этой страницы. Всего: 101681
2025-03-28 14:41:53,867 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:53,874 - INFO - Добавлено 500 записей с этой страницы. Всего: 102181
2025-03-28 14:41:59,419 - INFO - Получено 500 записей на странице.
2025-03-28 14:41:59,426 - INFO - Добавлено 500 записей с этой страницы. Всего: 1026

...

2025-03-28 17:52:26,615 - INFO - Получено 500 записей на странице.
2025-03-28 17:52:26,628 - INFO - Добавлено 500 записей с этой страницы. Всего: 742809
2025-03-28 17:52:32,028 - INFO - Получено 500 записей на странице.
2025-03-28 17:52:32,041 - INFO - Добавлено 500 записей с этой страницы. Всего: 743309
2025-03-28 17:53:07,145 - ERROR - Таймаут запроса, ждём 30 секунд и пробуем снова...
2025-03-28 17:53:37,571 - INFO - Получено 500 записей на странице.
2025-03-28 17:53:37,585 - INFO - Добавлено 500 записей с этой страницы. Всего: 743809
2025-03-28 17:53:43,158 - INFO - Получено 500 записей на странице.
2025-03-28 17:53:43,168 - INFO - Добавлено 500 записей с этой страницы. Всего: 744309
2025-03-28 17:53:48,592 - INFO - Получено 500 записей на странице.
2025-03-28 17:53:48,607 - INFO - Добавлено 500 записей с этой страницы. Всего: 744809
2025-03-28 17:53:54,000 - INFO - Получено 500 записей на странице.
2025-03-28 17:53:54,014 - INFO - Добавлено 500 записей с этой страницы. Всего: 7453

2025-03-28 18:00:10,869 - INFO - Добавлено 500 записей с этой страницы. Всего: 767306
2025-03-28 18:00:16,363 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:16,377 - INFO - Добавлено 500 записей с этой страницы. Всего: 767806
2025-03-28 18:00:22,017 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:22,030 - INFO - Добавлено 500 записей с этой страницы. Всего: 768306
2025-03-28 18:00:27,563 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:27,577 - INFO - Добавлено 500 записей с этой страницы. Всего: 768806
2025-03-28 18:00:33,217 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:33,230 - INFO - Добавлено 500 записей с этой страницы. Всего: 769306
2025-03-28 18:00:38,674 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:38,683 - INFO - Добавлено 440 записей с этой страницы. Всего: 769746
2025-03-28 18:00:44,172 - INFO - Получено 500 записей на странице.
2025-03-28 18:00:44,187 - INFO - Добавлено 500 записей с этой страницы. Всего: 770

2025-03-28 18:09:44,601 - INFO - Получено 500 записей на странице.
2025-03-28 18:09:44,615 - INFO - Добавлено 500 записей с этой страницы. Всего: 791745
2025-03-28 18:09:44,653 - INFO - Сохранён временный файл: temp_parquet_files\temp_batch_75.parquet с 10000 записями
2025-03-28 18:09:50,345 - INFO - Получено 500 записей на странице.
2025-03-28 18:09:50,359 - INFO - Добавлено 500 записей с этой страницы. Всего: 792245
2025-03-28 18:09:55,808 - INFO - Получено 500 записей на странице.
2025-03-28 18:09:55,822 - INFO - Добавлено 500 записей с этой страницы. Всего: 792745
2025-03-28 18:10:01,209 - INFO - Получено 500 записей на странице.
2025-03-28 18:10:01,223 - INFO - Добавлено 500 записей с этой страницы. Всего: 793245
2025-03-28 18:10:06,670 - INFO - Получено 500 записей на странице.
2025-03-28 18:10:06,684 - INFO - Добавлено 500 записей с этой страницы. Всего: 793745
2025-03-28 18:10:12,087 - INFO - Получено 500 записей на странице.
2025-03-28 18:10:12,105 - INFO - Добавлено 500 запис

2025-03-28 18:15:21,702 - INFO - Добавлено 500 записей с этой страницы. Всего: 816745
2025-03-28 18:15:27,073 - INFO - Получено 500 записей на странице.
2025-03-28 18:15:27,087 - INFO - Добавлено 500 записей с этой страницы. Всего: 817245
2025-03-28 18:15:32,483 - INFO - Получено 500 записей на странице.
2025-03-28 18:15:32,495 - INFO - Добавлено 500 записей с этой страницы. Всего: 817745
2025-03-28 18:16:08,019 - ERROR - Ошибка соединения: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out., ждём 30 секунд и пробуем снова...
2025-03-28 18:16:38,504 - INFO - Получено 500 записей на странице.
2025-03-28 18:16:38,517 - INFO - Добавлено 500 записей с этой страницы. Всего: 818245
2025-03-28 18:16:43,973 - INFO - Получено 500 записей на странице.
2025-03-28 18:16:43,986 - INFO - Добавлено 500 записей с этой страницы. Всего: 818745
2025-03-28 18:16:49,453 - INFO - Получено 500 записей на странице.
2025-03-28 18:16:49,466 - INFO - Добавлено 500 записей с этой страницы. 

2025-03-28 18:22:53,588 - INFO - Добавлено 500 записей с этой страницы. Всего: 841245
2025-03-28 18:22:58,900 - INFO - Получено 500 записей на странице.
2025-03-28 18:22:58,912 - INFO - Добавлено 500 записей с этой страницы. Всего: 841745
2025-03-28 18:22:58,951 - INFO - Сохранён временный файл: temp_parquet_files\temp_batch_80.parquet с 10000 записями
2025-03-28 18:23:04,483 - INFO - Получено 500 записей на странице.
2025-03-28 18:23:04,497 - INFO - Добавлено 500 записей с этой страницы. Всего: 842245
2025-03-28 18:23:09,917 - INFO - Получено 500 записей на странице.
2025-03-28 18:23:09,931 - INFO - Добавлено 500 записей с этой страницы. Всего: 842745
2025-03-28 18:23:15,355 - INFO - Получено 500 записей на странице.
2025-03-28 18:23:15,369 - INFO - Добавлено 500 записей с этой страницы. Всего: 843245
2025-03-28 18:23:20,785 - INFO - Получено 500 записей на странице.
2025-03-28 18:23:20,799 - INFO - Добавлено 500 записей с этой страницы. Всего: 843745
2025-03-28 18:23:26,405 - INFO - 